In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALSModel
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder

import numpy as np
from numpy import linalg
from numpy.linalg import norm
from scipy.spatial.distance import squareform, pdist

# We import sklearn.
import sklearn
from sklearn.manifold import TSNE
from sklearn.preprocessing import scale

# We'll hack a bit with the t-SNE code in sklearn 0.15.2.
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.manifold.t_sne import (_joint_probabilities,
									_kl_divergence)
# from sklearn.utils.extmath import _ravel
# Random state.
RS = 20150101

# We'll use matplotlib for graphics.
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import matplotlib

# # We import seaborn to make nice plots.
import seaborn as sns
sns.set_style('darkgrid')
sns.set_palette('muted')
sns.set_context("notebook", font_scale=1.5,
				rc={"lines.linewidth": 2.5})

# # We'll generate an animation with matplotlib and moviepy.
# from moviepy.video.io.bindings import mplfig_to_npimage
# import moviepy.editor as mpy
import argparse

In [2]:
# palette = np.array(sns.color_palette("hls", 4))
# d = {0:0, 1:1, 8:2, 9:3}
# new_y=[]
# for v in y:
#     if v in d:
#         new_y.append(d[v])
   

In [3]:
# def scatter(x, colors):
# 	# We choose a color palette with seaborn.
# 	palette = np.array(sns.color_palette("hls", 4))

# 	# We create a scatter plot.
# 	f = plt.figure(figsize=(8, 8))
# 	ax = plt.subplot(aspect='equal')
# 	x_new = x[np.isin(colors, [0,1,8,9])]

# 	sc = ax.scatter(x_new[:,0], x_new[:,1], lw=0, s=40,c=palette[np.array(new_y).astype(np.int)])
# 	plt.xlim(-25, 25)
# 	plt.ylim(-25, 25)
# 	ax.axis('off')
# 	ax.axis('tight')
    

# 	# We add the labels for each digit.
# 	txts = []
# 	max_label = max(colors)
# # 	for i in range(max_label):
# 	for i in [0,1,8,9]:
# 		# Position of each label.
# 		xtext, ytext = np.median(x[colors == i, :], axis=0)
# 		txt = ax.text(xtext, ytext, mapping[i], fontsize=10)
# 		txt.set_path_effects([
# 			PathEffects.Stroke(linewidth=5, foreground="w"),
# 			PathEffects.Normal()])
# 		txts.append(txt)

# 	return f, ax, sc, txts

In [4]:
import pandas as pd
final_df = pd.read_csv('best_model100.0_version2.0final_df.csv')

In [ ]:
# # SELECT SOME GENRES
# l = ['children','comics, graphic','young-adult','romance']
# final_df = final_df[final_df.genres.isin(l)]

In [ ]:
le = LabelEncoder()
final_df['label'] = le.fit_transform(list(final_df.genres))
mapping = dict(zip( range(len(le.classes_)),le.classes_))

In [ ]:
final_df.head(5)

,Unnamed: 0,bid_real,bid_model,genres,0,1,2,3,4,5,...,81,82,83,84,85,86,87,88,89,label
0,0,10023211,746472,non-fiction,0.330932,0.467727,0.121590,0.046551,0.116730,0.282812,...,0.322478,0.053955,0.488816,0.160073,0.656364,0.024673,0.000000,0.000000,0.213062,6
1,1,1002887,1493252,non-fiction,0.000000,0.075000,0.000000,0.048752,0.067509,0.000000,...,0.149306,0.000000,0.056689,0.000000,0.399515,0.000141,0.176206,0.177200,0.013110,6
2,2,10046808,1843492,children,0.000000,0.000000,0.000000,0.050474,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.056111,0.730813,0.000000,0.000000,0.000000,0.644091,0
3,3,10051877,750238,non-fiction,0.000000,0.000000,0.000000,0.308894,0.000000,0.000000,...,0.000000,0.000000,0.108588,0.063941,0.000000,0.000000,0.093702,0.000000,0.000000,6
4,4,10054654,814558,young-adult,0.000000,0.087331,0.092462,0.108800,0.000000,0.061165,...,0.137286,0.082138,0.071623,0.000000,0.032869,0.328303,0.166225,0.410127,0.112220,9


In [ ]:
mapping

{0: 'children',
 1: 'comics, graphic',
 2: 'fantasy, paranormal',
 3: 'fiction',
 4: 'history, historical fiction, biography',
 5: 'mystery, thriller, crime',
 6: 'non-fiction',
 7: 'poetry',
 8: 'romance',
 9: 'young-adult'}

In [ ]:
def scatter(x, colors):
    # We choose a color palette with seaborn.
    palette = np.array(sns.color_palette("hls", 10))

    # We create a scatter plot.
    f = plt.figure(figsize=(20, 20))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=40,
                    c=palette[colors.astype(np.int)])
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')

    # We add the labels for each digit.
    txts = []
    for i in range(10):
        # Position of each label.
        xtext, ytext = np.median(x[colors == i, :], axis=0)
        txt = ax.text(xtext, ytext, mapping[i], fontsize=24)
        txt.set_path_effects([
            PathEffects.Stroke(linewidth=5, foreground="w"),
            PathEffects.Normal()])
        txts.append(txt)

    return f, ax, sc, txts

In [ ]:
# We first reorder the data points according to the labels
len_label = len(le.classes_)
X = np.vstack([final_df[[ str(i) for i in range(90)]][final_df.label==i]
               for i in range(len_label)])
y = np.hstack([final_df.label[final_df.label==i]
           for i in range(len_label)])
digits_proj = TSNE(random_state=RS).fit_transform(X)

f, ax, sc, txts = scatter(digits_proj, y)
# f.savefig(filepath+'figure.png')

In [ ]:
mapping